In [1]:
from sklearn.mixture import GaussianMixture

## SAVIC-C - C

### load SAVIC-C - C GM Classifier

In [2]:
gmm_c_name = 'Output/ML/models/GMM_C/GMM_C'

# reload
means = np.load(gmm_c_name + '_means.npy')
covar = np.load(gmm_c_name + '_covariances.npy')
gmm_c = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_c.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_c.weights_ = np.load(gmm_c_name + '_weights.npy')
gmm_c.means_ = means
gmm_c.covariances_ = covar



### load SAVIC-C - C processing function

In [3]:
def SAVIC_C_C(df_c_):
    
    df_c_uns_ = df_c_[df_c_['unstable']].drop(columns = ['unstable'])
    df_c_uns_.drop(columns = ['Pc'], inplace = True)
    
    df_c_uns_pre_SC_ = df_c_uns_.copy()
    df_c_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_c_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_c_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_c_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_c_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_c_uns_pre_SC_['alph_c']
    df_c_uns_pre_SC_['uns_core'] = df_c_uns_pre_SC_['uns_OFH'] | df_c_uns_pre_SC_['uns_FM'] | df_c_uns_pre_SC_['uns_Mirror'] | df_c_uns_pre_SC_['uns_IC']
    df_c_uns_pre_SC_.drop(columns = ['uns_Mirror'], inplace = True)
    
    df_c_uns_pre_SC_[['beta_par_core', 'alph_c']] = np.log10(df_c_uns_pre_SC_[['beta_par_core', 'alph_c']])
    df_c_uns_pre_SC_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_c_uns_post_SC_ = df_c_uns_pre_SC_.copy()
    df_c_uns_post_SC_['cluster'] = gmm_c.predict(df_c_uns_pre_SC_[['log_beta_par_core', 'log_alph_c', 'theta_kB', \
                                                                   'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core']].values)
    
    df_c_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_c_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_c_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_c_uns_post_SC_['ins_type'] = np.nan
    ins_types = ['Ion Cyclotron', 'Parallel Firehose', 'Mirror', 'Oblique Firehose']
    for cluster, ins_type in zip(np.arange(4), ins_types):
        df_c_uns_post_SC_['ins_type'][df_c_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_c_post_SC_ = df_c_.copy()
    df_c_post_SC_['ins_type'] = np.nan
    df_c_post_SC_.loc[df_c_uns_post_SC_.index, ['ins_type']] = df_c_uns_post_SC_['ins_type']
    
    return df_c_post_SC_
    

### run SAVIC-C - C 

In [4]:
df_c_uns_post_SC = SAVIC_C_C(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_C_post_Q'))
df_c_uns_post_SC

,beta_par_core,alph_c,unstable,Pc,theta_kB,ins_type
0,1.0,1.0,False,NaN,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113,Ion Cyclotron
2,1.0,0.4,True,0.000287,0.002231,Parallel Firehose
3,12.0,1.2,True,0.001800,0.001649,Ion Cyclotron


## SAVIC-C - CB

### load SAVIC-C - CB GM Classifier

In [5]:
gmm_cb_name = 'Output/ML/models/GMM_CB/GMM_CB'
means = np.load(gmm_cb_name + '_means.npy')
covar = np.load(gmm_cb_name + '_covariances.npy')
gmm_cb = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cb.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cb.weights_ = np.load(gmm_cb_name + '_weights.npy')
gmm_cb.means_ = means
gmm_cb.covariances_ = covar


### load SAVIC-C - CB processing function

In [6]:
def SAVIC_C_CB(df_cb_):
    
    df_cb_uns_ = df_cb_[df_cb_['unstable']].drop(columns = ['unstable'])
    df_cb_uns_pre_SC_ = df_cb_uns_.copy()

    df_cb_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_core'] = df_cb_uns_pre_SC_['uns_OFH'] | df_cb_uns_pre_SC_['uns_FM'] | df_cb_uns_pre_SC_['uns_Mirror'] | df_cb_uns_pre_SC_['uns_IC']

    df_cb_uns_pre_SC_['beta_par_beam'] = df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['D_b'] / df_cb_uns_pre_SC_['tau_b']
    df_cb_uns_pre_SC_['lambda_FH_beta'] = ( (df_cb_uns_pre_SC_['beta_par_core'] + df_cb_uns_pre_SC_['beta_par_beam']) - \
    (df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['alph_c'] + df_cb_uns_pre_SC_['beta_par_beam'] * df_cb_uns_pre_SC_['alph_b']) ) / 2.
    df_cb_uns_pre_SC_['lambda_FH_beam'] = df_cb_uns_pre_SC_['D_b'] * df_cb_uns_pre_SC_['vv_b']**2.
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH_beta'] + df_cb_uns_pre_SC_['lambda_FH_beam']# + df_cb_uns_pre_SC_['lambda_FH_alpha']
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH'] > 1.

    df_cb_uns_pre_SC_['Pow_core'][df_cb_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cb_uns_pre_SC_['Pow_beam'][df_cb_uns_pre_SC_['Pow_beam'] < 0] = 0

    
    column_list_cb_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cb_uns_pre_SC_cut_ = df_cb_uns_pre_SC_[column_list_cb_]
    
    df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cb_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_ = df_cb_uns_pre_SC_cut_.copy()
    df_cb_uns_post_SC_['cluster'] = gmm_cb.predict(df_cb_uns_pre_SC_cut_.values)
    
    df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cb_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_['ins_type'] = np.nan
    ins_types_cb_ = ['Oblique FM (B); resonant with Core', 'IC (B); $T_\perp/T_{||} < 1$', 'Oblique FM (B)', \
                    'Oblique Firehose', 'IC (B); $T_\perp/T_{||} > 1$', 'Parallel Firehose', 'IC (B), unstable core', 'IC (C)']
    for cluster, ins_type in zip(np.arange(8), ins_types_cb_):
        df_cb_uns_post_SC_['ins_type'][df_cb_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cb_post_SC_ = df_cb_.copy()
    df_cb_post_SC_['ins_type'] = np.nan
    df_cb_post_SC_.loc[df_cb_uns_post_SC_.index, ['ins_type']] = df_cb_uns_post_SC_['ins_type']
    
    return df_cb_post_SC_

### run SAVIC-C - CB 

In [7]:
df_cb_post_SC = SAVIC_C_CB(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CB_post_Q'))
df_cb_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle,ins_type
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137,"IC (B), unstable core"
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414,IC (B); $T_\perp/T_{||} > 1$
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991,"IC (B), unstable core"


## SAVIC-C - core \& $\alpha$

### load SAVIC-C - C$\alpha$ GM Classifier

In [8]:
gmm_ca_name = 'Output/ML/models/GMM_CA/GMM_CA'
means = np.load(gmm_ca_name + '_means.npy')
covar = np.load(gmm_ca_name + '_covariances.npy')
gmm_ca = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_ca.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_ca.weights_ = np.load(gmm_ca_name + '_weights.npy')
gmm_ca.means_ = means
gmm_ca.covariances_ = covar

### load SAVIC-C - C$\alpha$ processing function

In [9]:
def SAVIC_C_CA(df_ca_):
    
    df_ca_uns_ = df_ca_[df_ca_['unstable']].drop(columns = ['unstable'])
    
    df_ca_uns_pre_SC_ = df_ca_uns_.copy()

    df_ca_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_core'] = df_ca_uns_pre_SC_['uns_OFH'] | df_ca_uns_pre_SC_['uns_FM'] | df_ca_uns_pre_SC_['uns_Mirror'] | df_ca_uns_pre_SC_['uns_IC']

    df_ca_uns_pre_SC_['beta_par_alpha'] = df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['D_a'] / df_ca_uns_pre_SC_['tau_a']
    df_ca_uns_pre_SC_['lambda_FH_beta'] = ( (df_ca_uns_pre_SC_['beta_par_core'] + df_ca_uns_pre_SC_['beta_par_alpha']) - \
    (df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['alph_c'] + df_ca_uns_pre_SC_['beta_par_alpha'] * df_ca_uns_pre_SC_['alph_a']) ) / 2.
    df_ca_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_ca_uns_pre_SC_['D_a'] / (1. + df_ca_uns_pre_SC_['D_a'] + 4 * df_ca_uns_pre_SC_['D_a']) * df_ca_uns_pre_SC_['vv_a']**2.
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH_beta'] + df_ca_uns_pre_SC_['lambda_FH_alpha']
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH'] > 1.

    df_ca_uns_pre_SC_['Pow_core'][df_ca_uns_pre_SC_['Pow_core'] < 0] = 0
    df_ca_uns_pre_SC_['Pow_alpha'][df_ca_uns_pre_SC_['Pow_alpha'] < 0] = 0
    
    column_list_ca_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_ca_uns_pre_SC_cut_ = df_ca_uns_pre_SC_[column_list_ca_]
    
    df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_ca_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_ca_uns_post_SC_ = df_ca_uns_pre_SC_cut_.copy()
    df_ca_uns_post_SC_['cluster'] = gmm_ca.predict(df_ca_uns_pre_SC_cut_.values)
    
    df_ca_uns_post_SC_['ins_type'] = np.nan
    ins_types_ca_ = ['Parallel Firehose', 'IC (A)', 'Oblique Firehose', \
                     'CGL Firehose; Mirror', 'A anis; borderline PFH', 'IC (C)']
    for cluster, ins_type in zip(np.arange(6), ins_types_ca_):
        df_ca_uns_post_SC_['ins_type'][df_ca_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_ca_post_SC_ = df_ca_.copy()
    df_ca_post_SC_['ins_type'] = np.nan
    df_ca_post_SC_.loc[df_ca_uns_post_SC_.index, ['ins_type']] = df_ca_uns_post_SC_['ins_type']
    
    return df_ca_post_SC_

### run SAVIC-C - C$\alpha$ 

In [10]:
df_ca_post_SP = SAVIC_C_CA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CA_post_Q'))
df_ca_post_SP

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962,Parallel Firehose
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290,IC (A)
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381,CGL Firehose; Mirror
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031,IC (C)


## SAVIC-C - core, beam \& $\alpha$

### load SAVIC-C - CB$\alpha$ XGB classifier 

In [11]:
gmm_cba_name = 'Output/ML/models/GMM_CBA/GMM_CBA'
means = np.load(gmm_cba_name + '_means.npy')
covar = np.load(gmm_cba_name + '_covariances.npy')
gmm_cba = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cba.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cba.weights_ = np.load(gmm_cba_name + '_weights.npy')
gmm_cba.means_ = means
gmm_cba.covariances_ = covar

### load SAVIC-C - CB$\alpha$ processing function

In [12]:
def SAVIC_C_CBA(df_cba_):
    df_cba_uns_ = df_cba_[df_cba_['unstable']].drop(columns = ['unstable'])
    
    df_cba_uns_pre_SC_ = df_cba_uns_.copy()

    df_cba_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_core'] = df_cba_uns_pre_SC_['uns_OFH'] | df_cba_uns_pre_SC_['uns_FM'] | df_cba_uns_pre_SC_['uns_Mirror'] | df_cba_uns_pre_SC_['uns_IC']


    df_cba_uns_pre_SC_['beta_par_beam'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_b'] / df_cba_uns_pre_SC_['tau_b']
    df_cba_uns_pre_SC_['beta_par_alpha'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_a'] / df_cba_uns_pre_SC_['tau_a']

    df_cba_uns_pre_SC_['lambda_FH_beta'] = ( (df_cba_uns_pre_SC_['beta_par_core'] + df_cba_uns_pre_SC_['beta_par_beam'] + df_cba_uns_pre_SC_['beta_par_alpha']) - \
    (df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['alph_c'] + df_cba_uns_pre_SC_['beta_par_beam'] * df_cba_uns_pre_SC_['alph_b'] + \
    df_cba_uns_pre_SC_['beta_par_alpha'] * df_cba_uns_pre_SC_['alph_a']) ) / 2.

    df_cba_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_cba_uns_pre_SC_['D_a'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_a']**2.
    df_cba_uns_pre_SC_['lambda_FH_beam'] = df_cba_uns_pre_SC_['D_b'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_b']**2.
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH_beta'] + df_cba_uns_pre_SC_['lambda_FH_alpha'] + df_cba_uns_pre_SC_['lambda_FH_beam']
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH'] > 1.

    df_cba_uns_pre_SC_['Pow_core'][df_cba_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_beam'][df_cba_uns_pre_SC_['Pow_beam'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_alpha'][df_cba_uns_pre_SC_['Pow_alpha'] < 0] = 0

    column_list_cba_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                        'tau_b', 'alph_b', 'D_b', 'vv_b', \
                        'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', \
                        'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cba_uns_pre_SC_cut_ = df_cba_uns_pre_SC_[column_list_cba_]
    
    df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cba_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_ = df_cba_uns_pre_SC_cut_.copy()
    df_cba_uns_post_SC_['cluster'] = gmm_cba.predict(df_cba_uns_pre_SC_cut_.values)
    
    df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cba_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_['r_cluster'] = np.nan
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 0] = 9
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 1] = 2
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 2] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 3] = 4
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 4] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 5] = 5
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 6] = 12
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 7] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 8] = 11
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 9] = 10
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 10] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 11] = 6
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 12] = 8
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 13] = 3

    df_cba_uns_post_SC_['r_cluster'] = df_cba_uns_post_SC_['r_cluster'].astype(int)
    
    df_cba_uns_post_SC_['ins_type'] = np.nan
    ins_types_cba_ = ['IC (C)', 'IC (C); A unstable', \
                      'IC (B), C unstable', 'IC (B); A unstable', 'IC (B); high B anis', 'IC (B); borderline PFH', \
                      'IC (A)', 'IC (A); C absorbing', \
                      'Oblique Firehose', 'Parallel Firehose', \
                      'FM (B), oblique', 'FM (B), oblique; mirror']
    for cluster, ins_type in zip(np.arange(12), ins_types_cba_):
        df_cba_uns_post_SC_['ins_type'][df_cba_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cba_post_SC_ = df_cba_.copy()
    df_cba_post_SC_['ins_type'] = np.nan
    df_cba_post_SC_.loc[df_cba_uns_post_SC_.index, ['ins_type']] = df_cba_uns_post_SC_['ins_type']
    
    return df_cba_post_SC_

### run SAVIC-C - CB$\alpha$ 

In [13]:
df_cba_post_SC = SAVIC_C_CBA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-Q_CBA_post_Q'))
df_cba_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945,Parallel Firehose
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523,IC (B); high B anis
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767,Oblique Firehose
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413,IC (C)
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371,Oblique Firehose
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954,IC (A); C absorbing
